In [30]:
    import pandas as pd

In [31]:
df_items = pd.read_parquet('../data/interim/parquet/final_items.parquet')
df_games = pd.read_parquet('../data/interim/parquet/final_games.parquet')
df_reviews = pd.read_parquet('../data/interim/parquet/final_reviews.parquet')

In [32]:
df_items['item_id'] = df_items['item_id'].astype(int)

In [33]:

# Asumiendo que df_items, df_reviews y df_games son tus DataFrames.

# Primer merge: unir df_items y df_games en 'item_id' y 'id'
merged_df_1 = pd.merge(df_items, df_games, left_on='item_id', right_on='id')

# Segundo merge: unir el DataFrame resultante con df_reviews en 'user_id'
final_merged_df = pd.merge(merged_df_1, df_reviews, on='user_id')



In [34]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6562912 entries, 0 to 6562911
Data columns (total 51 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   user_id                object 
 1   steam_id               object 
 2   item_id_x              int32  
 3   item_name              object 
 4   playtime_forever       int64  
 5   app_name               object 
 6   id                     int32  
 7   price                  float64
 8   release_year           int32  
 9   Indie                  int32  
 10  Action                 int32  
 11  Adventure              int32  
 12  Casual                 int32  
 13  Simulation             int32  
 14  Strategy               int32  
 15  RPG                    int32  
 16  Singleplayer           int32  
 17  Free to Play           int32  
 18  Multiplayer            int32  
 19  Great Soundtrack       int32  
 20  Puzzle                 int32  
 21  Early Access           int32  
 22  2D                

In [35]:
# Definir el nuevo orden de las columnas
new_column_order = [
    'user_id', 'steam_id', 'item_id_x', 'item_name', 'playtime_forever',
    'app_name', 'id', 'price', 'release_year', 'funny', 'posted',
    'item_id_y', 'helpful', 'recommend', 'sentiment_analysis',
    # Mover las columnas de géneros al final
    'Indie', 'Action', 'Adventure', 'Casual', 'Simulation', 'Strategy', 'RPG',
    'Singleplayer', 'Free to Play', 'Multiplayer', 'Great Soundtrack', 'Puzzle',
    'Early Access', '2D', 'Atmospheric', 'VR', 'Sports', 'Platformer',
    'Story Rich', 'Sci-fi', 'Fantasy', 'Horror', 'Open World', 'Difficult',
    'Anime', 'Massively Multiplayer', 'Pixel Graphics', 'Co-op', 'Shooter',
    'Racing', 'Female Protagonist', 'Funny', 'First-Person', 'FPS', 'Sandbox',
    'Arcade'
]

# Reorganizar las columnas
final_merged_df = final_merged_df.reindex(columns=new_column_order)


In [36]:
# Seleccionar las columnas relevantes
genre_playtime_df = final_merged_df[['user_id', 'playtime_forever', 'release_year', 'Indie', 'Action', 'Adventure', 'Casual', 'Simulation', 'Strategy', 'RPG', 'Singleplayer', 'Free to Play', 'Multiplayer', 'Great Soundtrack', 'Puzzle', 'Early Access', '2D', 'Atmospheric', 'VR', 'Sports', 'Platformer', 'Story Rich', 'Sci-fi', 'Fantasy', 'Horror', 'Open World', 'Difficult', 'Anime', 'Massively Multiplayer', 'Pixel Graphics', 'Co-op', 'Shooter', 'Racing', 'Female Protagonist', 'Funny', 'First-Person', 'FPS', 'Sandbox', 'Arcade']]

genre_playtime_df = genre_playtime_df.copy()
genre_playtime_df['release_year'] = genre_playtime_df['release_year'].astype(int)

# Guardar el DataFrame
genre_playtime_df.to_parquet('../data/processed/genre_playtime.parquet')


In [37]:
# Seleccionar las columnas relevantes
recommendations_df = final_merged_df[['app_name', 'release_year', 'recommend', 'sentiment_analysis']]

# Guardar el DataFrame
recommendations_df.to_parquet('../data/processed/recommendations.parquet')


In [38]:
def PlayTimeGenre(genero: str):
    # Cargar el DataFrame genre_playtime_df
    genre_playtime_df = pd.read_parquet('../data/processed/genre_playtime.parquet')
    
    # Filtrar solo las filas donde el género especificado es 1 (presente)
    genre_df = genre_playtime_df[genre_playtime_df[genero] == 1]
    
    # Agrupar por año de lanzamiento y sumar el tiempo de juego
    year_playtime = genre_df.groupby('release_year')['playtime_forever'].sum()
    
    # Encontrar el año con el máximo tiempo de juego
    max_playtime_year = year_playtime.idxmax()
    
    return {"Año de lanzamiento con más horas jugadas para Género " + genero : max_playtime_year}


In [39]:
PlayTimeGenre('Indie')

{'Año de lanzamiento con más horas jugadas para Género Indie': 2006}

In [40]:
def UserForGenre(genero: str):
    # Cargar el DataFrame genre_playtime_df
    genre_playtime_df = pd.read_parquet('../data/processed/genre_playtime.parquet')
    
    # Filtrar solo las filas donde el género especificado es 1 (presente)
    genre_df = genre_playtime_df[genre_playtime_df[genero] == 1]
    
    # Agrupar por usuario y sumar el tiempo de juego
    user_playtime = genre_df.groupby('user_id')['playtime_forever'].sum()
    
    # Encontrar el usuario con el máximo tiempo de juego
    max_playtime_user = user_playtime.idxmax()
    
    # Filtrar las filas para el usuario específico
    user_df = genre_df[genre_df['user_id'] == max_playtime_user]
    
    # Agrupar por año de lanzamiento y sumar el tiempo de juego
    user_year_playtime = user_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    
    # Crear la lista de horas jugadas por año
    hours_played = user_year_playtime.to_dict('records')
    
    return {"Usuario con más horas jugadas para Género " + genero : max_playtime_user, "Horas jugadas": hours_played}



In [41]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para Género Indie': '76561198063648921',
 'Horas jugadas': [{'release_year': -1, 'playtime_forever': 4368},
  {'release_year': 1999, 'playtime_forever': 12816},
  {'release_year': 2003, 'playtime_forever': 114576},
  {'release_year': 2006, 'playtime_forever': 6938448},
  {'release_year': 2010, 'playtime_forever': 23016},
  {'release_year': 2011, 'playtime_forever': 203520},
  {'release_year': 2012, 'playtime_forever': 181512},
  {'release_year': 2013, 'playtime_forever': 192768},
  {'release_year': 2014, 'playtime_forever': 354144},
  {'release_year': 2015, 'playtime_forever': 253632},
  {'release_year': 2016, 'playtime_forever': 348528},
  {'release_year': 2017, 'playtime_forever': 12528}]}

In [42]:
def UsersRecommend(año: int):
    # Cargar el DataFrame recommendations_df
    recommendations_df = pd.read_parquet('../data/processed/recommendations.parquet')
    
    # Filtrar las filas para el año especificado y las recomendaciones positivas
    year_recommend_df = recommendations_df[(recommendations_df['release_year'] == año) & (recommendations_df['recommend'] == True)]
    
    # Agrupar por nombre de la aplicación y contar las recomendaciones
    game_recommend_count = year_recommend_df.groupby('app_name').size()
    
    # Ordenar los juegos por la cantidad de recomendaciones y tomar los 3 primeros
    top_3_games = game_recommend_count.sort_values(ascending=False).head(3).index.tolist()
    
    return [{"Puesto 1": top_3_games[0]}, {"Puesto 2": top_3_games[1]}, {"Puesto 3": top_3_games[2]}]


In [43]:
UsersRecommend(2018)

[{'Puesto 1': 'Lost Moon'},
 {'Puesto 2': 'Divinia Chronicles: Relics of Gan-Ti'},
 {'Puesto 3': 'Empires Of Creation'}]

In [44]:
def UsersNotRecommend(año: int):
    # Reutilizar la función UsersRecommend, pero con el filtro de recomendación negativa
    recommendations_df = pd.read_parquet('../data/processed/recommendations.parquet')
    year_not_recommend_df = recommendations_df[(recommendations_df['release_year'] == año) & (recommendations_df['recommend'] == False)]
    game_not_recommend_count = year_not_recommend_df.groupby('app_name').size()
    bottom_3_games = game_not_recommend_count.sort_values().head(3).index.tolist()
    return [{"Puesto 1": bottom_3_games[0]}, {"Puesto 2": bottom_3_games[1]}, {"Puesto 3": bottom_3_games[2]}]


In [45]:
UsersNotRecommend(2001)

[{'Puesto 1': 'The Mystery of the Druids'},
 {'Puesto 2': "Kohan: Ahriman's Gift"},
 {'Puesto 3': 'Kohan: Immortal Sovereigns'}]

In [46]:
def sentiment_analysis(año: int):
    # Cargar el DataFrame recommendations_df
    recommendations_df = pd.read_parquet('../data/processed/recommendations.parquet')
    
    # Filtrar las filas para el año especificado
    year_sentiment_df = recommendations_df[recommendations_df['release_year'] == año]
    
    # Contar los registros de análisis de sentimiento
    sentiment_counts = year_sentiment_df['sentiment_analysis'].value_counts()
    
    return {"Negative": sentiment_counts.get(0, 0),
            "Neutral": sentiment_counts.get(1, 0),
            "Positive": sentiment_counts.get(2, 0)}


In [47]:
sentiment_analysis(2008)

{'Negative': 30802, 'Neutral': 52746, 'Positive': 85174}

In [48]:
# Eliminar la columna item_id_y y renombrar item_id_x a item_id
final_merged_df = final_merged_df.drop(columns=['item_id_y']).rename(columns={'item_id_x': 'item_id'})

# Ahora final_merged_df tiene toda la información combinada de los tres datasets

final_merged_df.to_parquet('../data/processed/merge.parquet', index=False)
final_merged_df.to_csv('../data/interim/csv/merge.csv', index=False)

In [49]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

def generate_features(df, group_by_col, feature_cols):
    feature_series_list = [df.groupby(group_by_col)[col].agg(['sum', 'mean']) for col in feature_cols]
    features_df = pd.concat(feature_series_list, axis=1)
    return features_df

def compute_cosine_similarity(features_df):
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features_df)
    cosine_sim_matrix = cosine_similarity(features_scaled, features_scaled)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=features_df.index, columns=features_df.index)
    return cosine_sim_df

# Seleccionar solo las columnas deseadas y eliminar duplicados
reduced_df = final_merged_df[['item_id', 'app_name', 'playtime_forever', 'recommend', 'sentiment_analysis']].drop_duplicates(subset='item_id')
reduced_df.to_parquet('../data/processed/reduced_df.parquet')

# Calcular características y similitud del coseno para juegos
game_features_df = generate_features(reduced_df, 'item_id', ['playtime_forever', 'recommend', 'sentiment_analysis'])
game_cosine_sim_df = compute_cosine_similarity(game_features_df)
game_cosine_sim_df.to_parquet('../data/processed/recomendacion_juego.parquet')

# Calcular características y similitud del coseno para usuarios
user_features_df = generate_features(final_merged_df, 'user_id', ['playtime_forever', 'sentiment_analysis', 'recommend'])
user_cosine_sim_df = compute_cosine_similarity(user_features_df)
user_cosine_sim_df.to_parquet('../data/processed/recomendacion_usuario.parquet')


In [50]:
# # Seleccionar solo las columnas deseadas
# reduced_df = final_merged_df[['item_id', 'app_name', 'playtime_forever', 'recommend', 'sentiment_analysis']]

# # Eliminar filas duplicadas basadas en la columna 'item_id'
# reduced_df = reduced_df.drop_duplicates(subset='item_id')

# # Guardar el DataFrame reducido en un archivo parquet
# reduced_df.to_parquet('../data/processed/parquet/recomendacion_juego.parquet')

# # Ahora, para el análisis de similitud del coseno:

# # Calcular las características por juego
# playtime_per_game = reduced_df.groupby('item_id')['playtime_forever'].sum()
# recommend_per_game = reduced_df.groupby('item_id')['recommend'].mean()
# sentiment_per_game = reduced_df.groupby('item_id')['sentiment_analysis'].mean()

# # Combinar todas las características en un solo DataFrame
# features_df = pd.concat([playtime_per_game, recommend_per_game, sentiment_per_game], axis=1)

# # Normalizar las características
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features_df)
# features_df_scaled = pd.DataFrame(features_scaled, index=features_df.index, columns=features_df.columns)

# # Calcular la matriz de similitud del coseno
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim_matrix = cosine_similarity(features_df_scaled, features_df_scaled)
# cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=features_df.index, columns=features_df.index)

# cosine_sim_df.to_parquet('../data/interim/parquet/parquet/recomendacion_usuario.parquet')


In [51]:

# Definir la función de recomendación basada en similitud del coseno
def recomendacion_juego(item_id):
    # Obtener las similitudes del juego dado
    sim_scores = game_cosine_sim_df[item_id]
    # Ordenar los juegos basados en la similitud del coseno
    recommended_games_ids = sim_scores.sort_values(ascending=False)[1:6].index  # Excluir el juego mismo
    # Buscar los nombres de los juegos recomendados
    recommended_games_names = reduced_df.set_index('item_id').loc[recommended_games_ids, 'app_name'].unique()
    # Devolver los nombres de los juegos recomendados
    return {"Juegos recomendados": recommended_games_names.tolist()}

# Prueba de la función de recomendación
recomendacion_juego(10)  # Asumiendo que 10 es un ID de juego válido


{'Juegos recomendados': ['Devouring Stars',
  'Platypus II',
  'Dino Run DX',
  'DICETINY: The Lord of the Dice',
  'Tennis Elbow 2013']}

In [52]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics.pairwise import cosine_similarity

# # Asumiendo que final_merged_df es tu DataFrame y ya has hecho la limpieza necesaria

# # Agregar las horas jugadas por usuario
# playtime_per_user = final_merged_df.groupby('user_id')['playtime_forever'].sum()

# # Agregar el análisis de sentimientos por usuario
# sentiment_per_user = final_merged_df.groupby('user_id')['sentiment_analysis'].mean()

# # Agregar las recomendaciones por usuario
# recommend_per_user = final_merged_df.groupby('user_id')['recommend'].mean()

# # Combinar en un solo DataFrame
# features_df = pd.concat([playtime_per_user, sentiment_per_user, recommend_per_user], axis=1)

# # Normalizar las características
# scaler = MinMaxScaler()
# features_scaled = scaler.fit_transform(features_df)
# features_df_scaled = pd.DataFrame(features_scaled, index=features_df.index, columns=features_df.columns)

# # Calcular la matriz de similitud del coseno
# cosine_sim_matrix = cosine_similarity(features_df_scaled, features_df_scaled)

# # Convertir la matriz de similitud del coseno en un DataFrame
# cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=features_df.index, columns=features_df.index)
# cosine_sim_df.to_parquet('../data/interim/parquet/parquetrecomendacion_usuario.parquet')


In [53]:
user_cosine_sim_df

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-I_AM_EPIC-,-Kenny,-Mad-,...,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c,zzoptimuszz
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,1.000000,0.948663,0.983849,0.977729,0.894375,0.999933,0.999847,0.948622,0.948662,0.948520,...,0.948639,0.999964,0.921456,0.907490,0.995146,0.948601,0.948582,0.999893,0.995435,0.948646
--ace--,0.948663,1.000000,0.989946,0.993857,0.707065,0.948675,0.948502,0.999984,0.999998,0.999882,...,0.999996,0.948575,0.996734,0.986332,0.973966,0.999982,0.999969,0.948658,0.971113,0.999996
--ionex--,0.983849,0.989946,1.000000,0.999459,0.799896,0.983860,0.983637,0.989938,0.989939,0.989791,...,0.989943,0.983757,0.975842,0.963428,0.995907,0.989927,0.989919,0.983840,0.994387,0.989946
-2SV-vuLB-Kg,0.977729,0.993857,0.999459,1.000000,0.780837,0.977751,0.977663,0.993817,0.993869,0.993849,...,0.993845,0.977640,0.982240,0.971297,0.992916,0.993836,0.993807,0.977740,0.991292,0.993841
-Azsael-,0.894375,0.707065,0.799896,0.780837,1.000000,0.894212,0.894504,0.706942,0.707097,0.707097,...,0.707002,0.894432,0.651091,0.636044,0.848089,0.706940,0.706888,0.894155,0.853074,0.707006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zwanzigdrei,0.948601,0.999982,0.989927,0.993836,0.706940,0.948682,0.948379,0.999995,0.999974,0.999855,...,0.999995,0.948463,0.996581,0.985924,0.973796,1.000000,0.999997,0.948682,0.970930,0.999992
zy0705,0.948582,0.999969,0.989919,0.993807,0.706888,0.948679,0.948321,0.999995,0.999956,0.999808,...,0.999988,0.948431,0.996513,0.985556,0.973717,0.999997,1.000000,0.948683,0.970810,0.999986
zynxgameth,0.999893,0.948658,0.983840,0.977740,0.894155,0.999995,0.999634,0.948678,0.948648,0.948523,...,0.948674,0.999734,0.921158,0.906810,0.994831,0.948682,0.948683,1.000000,0.995142,0.948672


In [54]:

def recomendacion_usuario(user_id: str):
    # Obtener las similitudes del usuario dado
    sim_scores = user_cosine_sim_df[user_id]
    # Ordenar los usuarios basados en la similitud del coseno
    similar_users = sim_scores.sort_values(ascending=False)[1:]  # Excluir el usuario mismo
    
    # Obtener los juegos jugados por los usuarios similares
    similar_users_list = similar_users.index.tolist()
    recommended_games_ids = final_merged_df[final_merged_df['user_id'].isin(similar_users_list)]['item_id'].unique()
    
    # Ordenar los juegos recomendados basados en la similitud del coseno
    recommended_games_ids = sorted(recommended_games_ids, key=lambda x: similar_users_list.index(final_merged_df[final_merged_df['item_id'] == x]['user_id'].iloc[0]))
    
    # Obtener los nombres de los 5 juegos más recomendados
    recommended_game_names = final_merged_df.set_index('item_id').loc[recommended_games_ids[:5], 'app_name'].unique()
    
    return {"Juegos recomendados": recommended_game_names.tolist()}


In [55]:

# Ejemplo de uso
recomendacion_usuario('zy0705')


{'Juegos recomendados': ['Liftoff',
  'Formula Truck 2013',
  'Shout Of Survival',
  'Aerofly FS 2 Flight Simulator',
  'Interplanetary']}

In [56]:
print(user_cosine_sim_df.index)
print(game_cosine_sim_df.index)

Index(['--000--', '--ace--', '--ionex--', '-2SV-vuLB-Kg', '-Azsael-',
       '-Beave-', '-GM-Dragon', '-I_AM_EPIC-', '-Kenny', '-Mad-',
       ...
       'zumpo', 'zunbae', 'zuzuga2003', 'zv_odd', 'zvanik', 'zwanzigdrei',
       'zy0705', 'zynxgameth', 'zyr0n1c', 'zzoptimuszz'],
      dtype='object', name='user_id', length=22576)
Index([    10,     20,     30,     40,     50,     60,     70,     80,    130,
          220,
       ...
       527340, 527440, 527510, 527520, 527810, 527890, 527900, 528580, 528660,
       530720],
      dtype='int32', name='item_id', length=8994)


In [57]:
print('item_id' in final_merged_df.columns)  # Esto debería imprimir True


True
